In [14]:
from datasets import load_dataset, load_from_disk
from transformers import AutoTokenizer, DefaultDataCollator, AutoModelForQuestionAnswering, TrainingArguments, Trainer
from pprint import pprint
from utils.preprocess import preprocess_squad_training, preprocess_squad_validation, compute_metrics

import torch
device = torch.device("cpu")

In [15]:
from huggingface_hub import notebook_login

notebook_login()

In [16]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML


def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(
        dataset
    ), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset) - 1)
        while pick in picks:
            pick = random.randint(0, len(dataset) - 1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(
                lambda x: [typ.feature.names[i] for i in x]
            )
    display(HTML(df.to_html()))

In [17]:
squad = load_from_disk("./datasets/squad")
squad_v2 = load_from_disk("./datasets/squad_v2")
coqa = load_from_disk("./datasets/coqa")

NOTE: Impossible to answer elements are represented with empty `text` and `answer_start` fields...

In [18]:
trained_checkpoint = "distilbert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(trained_checkpoint)
model = AutoModelForQuestionAnswering.from_pretrained(trained_checkpoint)

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
doc_stride = 128

train_dataset = squad_v2["train"].map(lambda x: preprocess_squad_training(x, tokenizer, doc_stride), batched=True, remove_columns=squad_v2["train"].column_names)

In [20]:
validation_dataset = squad_v2["validation"].map(
    lambda x : preprocess_squad_validation(x, tokenizer, 128),
    batched=True,
    remove_columns=squad_v2["validation"].column_names,
)
len(squad_v2["validation"]), len(validation_dataset)

(11873, 11974)

In [21]:
from transformers import TrainingArguments

args = TrainingArguments(
    "distilbert-finetuned-squad2",
    eval_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=False,
    push_to_hub=False,
)

from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer,
)

/var/folders/kf/9dhrf2c10hqcgvm080qlhzvw0000gn/T/ipykernel_34671/893827436.py:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [22]:
trainer.train()

Step,Training Loss


KeyboardInterrupt: 

In [ ]:
small_eval_set = squad_v2["validation"].select(range(500))

eval_set = small_eval_set.map(
    lambda x: preprocess_squad_validation(x, tokenizer, doc_stride),
    batched=True,
    remove_columns=squad_v2["validation"].column_names,
)

eval_set_for_model = eval_set.remove_columns(["example_id", "offset_mapping"])

from torch.utils.data import DataLoader

eval_set_for_model.set_format("torch")  # makes columns return torch tensors

dataloader = DataLoader(eval_set_for_model, batch_size=16)

device = torch.device("mps")
trained_model = AutoModelForQuestionAnswering.from_pretrained(trained_checkpoint).to(device)

for batch in dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = trained_model(**batch)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
start_logits = outputs.start_logits.cpu().numpy()
end_logits = outputs.end_logits.cpu().numpy()

In [ ]:
start_logits = outputs.start_logits
end_logits = outputs.end_logits
print(start_logits.shape, end_logits.shape)

torch.Size([4, 512]) torch.Size([4, 512])


In [ ]:
print(len(start_logits))  # should be equal to len(features)

4


In [ ]:
compute_metrics(start_logits, end_logits, eval_set, small_eval_set)

  0%|          | 0/500 [00:00<?, ?it/s]

IndexError: index 4 is out of bounds for axis 0 with size 4

In [ ]:
answers = example["answers"]
start_char = answers["answer_start"][0]
end_char = start_char + len(answers["text"][0])

sequence_ids = tokenized_example.sequence_ids()


In [ ]:
# the answers to each question will be in the latter ("context") part of tokenized_example.

In [ ]:
# Start token index of the current span in the text.
token_start_index = 0
while sequence_ids[token_start_index] != 1:
    token_start_index += 1

# End token index of the current span in the text.
token_end_index = len(tokenized_example["input_ids"][0]) - 1
while sequence_ids[token_end_index] != 1:
    token_end_index -= 1

# Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
offsets = tokenized_example["offset_mapping"][0]
if (
    offsets[token_start_index][0] <= start_char
    and offsets[token_end_index][1] >= end_char
):
    # Move the token_start_index and token_end_index to the two ends of the answer.
    # Note: we could go after the last offset if the answer is the last word (edge case).
    while (
        token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char
    ):
        token_start_index += 1
    start_position = token_start_index - 1
    while offsets[token_end_index][1] >= end_char:
        token_end_index -= 1
    end_position = token_end_index + 1
    # print(token_start_index, token_end_index)
    print(start_position, end_position)
else:
    print("The answer is not in this feature.")

18 19


In [ ]:
print(
    tokenizer.decode(
        tokenized_example["input_ids"][0][start_position : end_position + 1]
    )
)
print(answers["text"][0])

jay z
Jay Z


In [ ]:
# first_token_id = tokenized_example["input_ids"][0][0]
# offsets = tokenized_example["offset_mapping"][0][0]
# print(
#     tokenizer.convert_ids_to_tokens([first_token_id])[0],
#     example["question"][offsets[0] : offsets[1]],
# )


In [ ]:
# model = AutoModelForQuestionAnswering.from_pretrained('bert-base-uncased')
# model.to(device)
# data_collator = DefaultDataCollator()

In [ ]:
# training_args = TrainingArguments(
#     output_dir="bert-squad-test",
#     eval_strategy="epoch",
#     learning_rate=2e-5,
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=16,
#     num_train_epochs=3,
#     weight_decay=0.01,
#     push_to_hub=False,
# )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_squad["train"],
#     eval_dataset=tokenized_squad["validation"],
#     processing_class=tokenizer,
#     data_collator=data_collator,
# )


In [ ]:
# trainer.train()